In [3]:
from pathlib import Path
import pandas as pd
import anndata as ad
import numpy as np

# Path to your folder with CSVs
folder_path = '/home/workspace/private/CSVs'
# Get all CSV files
csv_files = list(Path(folder_path).glob('*.csv'))
print(f"Found {len(csv_files)} CSV files")
# List to store dataframes
all_dfs = []
# Read each CSV and add sample_id and batch_id based on filename
for csv_file in csv_files:
    print(f"Reading {csv_file.name}...")
    df = pd.read_csv(csv_file)
    
    # Extract from filename
    parts = csv_file.stem.split('_')
    batch_id = parts[0]
    sample_id = '_'.join(parts[:2]) if len(parts) >= 2 else parts[0]
    
    df['sample_id'] = sample_id
    df['batch_id'] = batch_id
    
    all_dfs.append(df)
    
# Combine all dataframes
combined_df = pd.concat(all_dfs, ignore_index=True)
print(f"\nTotal cells across all samples: {len(combined_df)}")

# Get marker columns for all three intensity types
mean_columns = [col for col in combined_df.columns if col.startswith('Mean Intensity - ')]
median_columns = [col for col in combined_df.columns if col.startswith('Median Intensity - ')]
std_columns = [col for col in combined_df.columns if col.startswith('Standard Deviation Intensity - ')]

print(f"\nFound {len(mean_columns)} mean intensity columns (before filtering)")

# Extract marker names
marker_names_raw = [col.replace('Mean Intensity - ', '') for col in mean_columns]

# Define markers to exclude
markers_to_exclude = ['CD123-IL3RAAF750','pSTAT3Y705-D3A7AF555']
'''
# Define marker groups to combine
marker_groups_to_combine = {
    'pSTAT3Y705-D3A7AF555': ['pSTAT3Y705-D3A7AF555', 'pSTAT3Y705-D3A7AF5555']  # Combine typo with correct
}
'''

# Define markers to remove completely (DAPI markers)
markers_to_remove = ['DAPI_R15', 'DAPI_AF_R01']
#['DAPI_R14', 'DAPI_R15', 'DAPI_AF_R01']

# Show what will be combined
print("\n➕ Marker groups to combine:")
for combined_name, group_markers in marker_groups_to_combine.items():
    print(f"  {combined_name}:")
    for m in group_markers:
        print(f"    - {m}")

# Filter out markers ending with ' (2)' OR in the exclusion list OR in the removal list
markers_to_keep_mask = [
    not name.endswith(' (2)') and name not in markers_to_exclude and name not in markers_to_remove
    for name in marker_names_raw
]

# Get the filtered marker names and columns
marker_names_filtered = [name for name, keep in zip(marker_names_raw, markers_to_keep_mask) if keep]
mean_columns_filtered = [col for col, keep in zip(mean_columns, markers_to_keep_mask) if keep]
median_columns_filtered = [col for col, keep in zip(median_columns, markers_to_keep_mask) if keep]
std_columns_filtered = [col for col, keep in zip(std_columns, markers_to_keep_mask) if keep]

# Report what was removed
removed_markers = [name for name, keep in zip(marker_names_raw, markers_to_keep_mask) if not keep]
if removed_markers:
    print(f"\n⚠️  Removed {len(removed_markers)} marker(s):")
    for marker in removed_markers:
        if marker.endswith(' (2)'):
            print(f"    - {marker} (duplicate)")
        elif marker in markers_to_exclude:
            print(f"    - {marker} (excluded)")
        elif marker in markers_to_remove:
            print(f"    - {marker} (removed DAPI)")

print(f"\nKept {len(mean_columns_filtered)} markers (before combining)")

# Now combine marker groups by SUMMING
print(f"\n➕ Summing marker groups:")
final_marker_names = []
final_mean_data = []
final_median_data = []
final_std_data = []

# Track which markers have been combined
markers_already_combined = set()
for group_markers in marker_groups_to_combine.values():
    markers_already_combined.update(group_markers)

# Process each marker
for i, marker_name in enumerate(marker_names_filtered):
    # Check if this marker should be combined
    combined = False
    for combined_name, group_markers in marker_groups_to_combine.items():
        if marker_name in group_markers:
            # Only process this group once (when we encounter the first marker in the group)
            if marker_name == group_markers[0]:
                print(f"\n  Summing {len(group_markers)} markers into '{combined_name}':")
                for m in group_markers:
                    print(f"    - {m}")
                
                # Find all columns for this group across the three intensity types
                group_mean_cols = []
                group_median_cols = []
                group_std_cols = []
                
                for m in group_markers:
                    if m in marker_names_filtered:
                        idx = marker_names_filtered.index(m)
                        group_mean_cols.append(mean_columns_filtered[idx])
                        group_median_cols.append(median_columns_filtered[idx])
                        group_std_cols.append(std_columns_filtered[idx])
                
                # Sum the values across all markers in the group
                if group_mean_cols:
                    mean_data = combined_df[group_mean_cols].values
                    median_data = combined_df[group_median_cols].values
                    std_data = combined_df[group_std_cols].values
                    
                    print(f"    ✓ Found and summed {len(group_mean_cols)} columns")
                    
                    final_marker_names.append(combined_name)
                    final_mean_data.append(np.sum(mean_data, axis=1))
                    final_median_data.append(np.sum(median_data, axis=1))
                    final_std_data.append(np.sum(std_data, axis=1))
                else:
                    print(f"    ⚠️  Warning: No columns found for this group!")
            
            combined = True
            break
    
    # If not part of a combined group, keep as is
    if not combined:
        final_marker_names.append(marker_name)
        final_mean_data.append(combined_df[mean_columns_filtered[i]].values)
        final_median_data.append(combined_df[median_columns_filtered[i]].values)
        final_std_data.append(combined_df[std_columns_filtered[i]].values)

# Convert lists to arrays
X_mean = np.column_stack(final_mean_data)
X_median = np.column_stack(final_median_data)
X_std = np.column_stack(final_std_data)

print(f"\n{'='*80}")
print("NaN VALUE FILTERING")
print(f"{'='*80}")

'''
# Check for NaN values in each marker across all intensity types
markers_with_nans = []
for i, marker in enumerate(final_marker_names):
    mean_nans = np.sum(np.isnan(X_mean[:, i]))
    median_nans = np.sum(np.isnan(X_median[:, i]))
    std_nans = np.sum(np.isnan(X_std[:, i]))
    total_nans = mean_nans + median_nans + std_nans
    
    if total_nans > 0:
        pct_mean = (mean_nans / len(X_mean[:, i])) * 100
        pct_median = (median_nans / len(X_median[:, i])) * 100
        pct_std = (std_nans / len(X_std[:, i])) * 100
        
        markers_with_nans.append({
            'marker': marker,
            'mean_nans': mean_nans,
            'median_nans': median_nans,
            'std_nans': std_nans,
            'total_nans': total_nans,
            'pct_mean': pct_mean,
            'pct_median': pct_median,
            'pct_std': pct_std
        })
        
        print(f"\n⚠️  Marker: {marker}")
        print(f"    Mean NaNs:   {mean_nans:>8,} ({pct_mean:>6.2f}%)")
        print(f"    Median NaNs: {median_nans:>8,} ({pct_median:>6.2f}%)")
        print(f"    Std NaNs:    {std_nans:>8,} ({pct_std:>6.2f}%)")

if markers_with_nans:
    print(f"\n{'='*80}")
    print(f"Found {len(markers_with_nans)} markers with NaN values")
    print(f"{'='*80}")
    
    # Decision: Remove markers that are mostly NaN (>90% in mean intensity)
    markers_to_remove_nan = [m['marker'] for m in markers_with_nans if m['pct_mean'] > 90]
    
    if markers_to_remove_nan:
        print(f"\n🗑️  Removing {len(markers_to_remove_nan)} markers with >90% NaN values:")
        for marker in markers_to_remove_nan:
            print(f"    - {marker}")
        
        # Filter out markers with >90% NaNs
        markers_to_keep_idx = [i for i, marker in enumerate(final_marker_names) 
                               if marker not in markers_to_remove_nan]
        
        X_mean = X_mean[:, markers_to_keep_idx]
        X_median = X_median[:, markers_to_keep_idx]
        X_std = X_std[:, markers_to_keep_idx]
        final_marker_names = [final_marker_names[i] for i in markers_to_keep_idx]
        
        print(f"\n✓ Markers remaining after NaN filtering: {len(final_marker_names)}")
    
    # For remaining markers with some NaNs, replace NaN with 0
    markers_with_partial_nans = [m['marker'] for m in markers_with_nans 
                                  if m['marker'] not in markers_to_remove_nan]
    
    if markers_with_partial_nans:
        print(f"\n🔧 Replacing NaN with 0 for {len(markers_with_partial_nans)} markers with partial NaN values:")
        for marker in markers_with_partial_nans:
            marker_idx = final_marker_names.index(marker)
            
            mean_nans_before = np.sum(np.isnan(X_mean[:, marker_idx]))
            median_nans_before = np.sum(np.isnan(X_median[:, marker_idx]))
            std_nans_before = np.sum(np.isnan(X_std[:, marker_idx]))
            
            # Replace NaN with 0
            X_mean[:, marker_idx] = np.nan_to_num(X_mean[:, marker_idx], nan=0.0)
            X_median[:, marker_idx] = np.nan_to_num(X_median[:, marker_idx], nan=0.0)
            X_std[:, marker_idx] = np.nan_to_num(X_std[:, marker_idx], nan=0.0)
            
            print(f"    - {marker}: {mean_nans_before + median_nans_before + std_nans_before:,} NaNs → 0")
else:
    print("\n✓ No NaN values found in any markers")

# Verify no NaNs remain
total_nans_remaining = np.sum(np.isnan(X_mean)) + np.sum(np.isnan(X_median)) + np.sum(np.isnan(X_std))
print(f"\n{'='*80}")
print(f"NaN verification: {total_nans_remaining} NaN values remaining across all data")
print(f"{'='*80}")
'''
'''
# Remove outliers using stricter IQR method (2.0 instead of 1.5) for each marker
print(f"\n{'='*80}")
print("Removing outliers using stricter IQR method (values > Q3 + 2.0*IQR)...")
print(f"{'='*80}")
for i, marker in enumerate(final_marker_names):
    # Calculate IQR and outlier threshold for each intensity type
    for X, name in [(X_mean, 'mean'), (X_median, 'median'), (X_std, 'std')]:
        q1 = np.percentile(X[:, i], 25)
        q3 = np.percentile(X[:, i], 75)
        iqr = q3 - q1
        upper_threshold = q3 + 2.0 * iqr  # Changed from 1.5 to 2.0
        
        # Count outliers before capping
        n_outliers = np.sum(X[:, i] > upper_threshold)
        
        # Cap values at upper threshold
        if name == 'mean':
            X_mean[:, i] = np.where(X_mean[:, i] > upper_threshold, upper_threshold, X_mean[:, i])
        elif name == 'median':
            X_median[:, i] = np.where(X_median[:, i] > upper_threshold, upper_threshold, X_median[:, i])
        else:
            X_std[:, i] = np.where(X_std[:, i] > upper_threshold, upper_threshold, X_std[:, i])
        
        if n_outliers > 0 and name == 'mean':  # Only print once per marker
            pct_outliers = (n_outliers / len(X[:, i])) * 100
            print(f"  {marker}: {n_outliers} cells ({pct_outliers:.2f}%) capped at {upper_threshold:.2f}")

print(f"\n{'='*80}")
print(f"Final marker count: {len(final_marker_names)}")
print(f"{'='*80}")
print(f"\nFinal markers:")
for m in final_marker_names:
    print(f"  - {m}")

'''

# Create obs DataFrame
obs = pd.DataFrame({
    'cell_id': combined_df['ID'].astype(str) + '_' + combined_df['sample_id'],
    'sample_id': combined_df['sample_id'],
    'Timeframe': combined_df['Timeframe'].astype(str),
    'batch_id': combined_df['batch_id'],
    'x': combined_df['Centroid X (µm)'],
    'y': combined_df['Centroid Y (µm)'],
    'Outline Area (µm²)': combined_df['Outline Area (µm²)'],
    'Perimeter (µm)': combined_df['Perimeter (µm)'],
    'Circularity': combined_df['Circularity']
})
obs.index = obs['cell_id']

# Create var DataFrame with clean marker names
var = pd.DataFrame({
    'marker_name': final_marker_names
})
var.index = final_marker_names

# Create AnnData object with mean as main matrix
adata = ad.AnnData(X=X_mean, obs=obs, var=var)

# Add all three intensity measures as layers
adata.layers['intensity_mean'] = X_mean
adata.layers['intensity_median'] = X_median
adata.layers['intensity_std'] = X_std

print(f"\n{'='*80}")
print("ANNDATA OBJECT SUMMARY")
print(f"{'='*80}")
print(adata)
print(f"\nLayers: {list(adata.layers.keys())}")
print(f"\nBatches: {adata.obs['batch_id'].unique()}")
print(f"\nSamples: {adata.obs['sample_id'].unique()}")
print(f"Number of cells: {adata.n_obs}")
print(f"Number of markers: {adata.n_vars}")

# Save the AnnData object
output_filename = 'combined_data_0020226.h5ad'
adata.write(output_filename)
print(f"\n✓ Saved to {output_filename}")
print(f"{'='*80}")

Found 50 CSV files
Reading MS00000915_R0_Cell Membrane.csv...
Reading MS00000915_R1_Cell Membrane.csv...
Reading MS00000917_R0_Cell Membrane.csv...
Reading MS00000917_R1_Cell Membrane.csv...
Reading MS00000919_R0_Cell Membrane.csv...
Reading MS00000919_R1_Cell Membrane.csv...
Reading MS00000921_R0_Cell Membrane.csv...
Reading MS00000921_R1_Cell Membrane.csv...
Reading MS00000923_R0_Cell Membrane.csv...
Reading MS00000923_R1_Cell Membrane.csv...
Reading MS00000925_R0_Cell Membrane.csv...
Reading MS00000925_R1_Cells_Cell Membrane.csv...
Reading MS00000927_R0_Cell Membrane.csv...
Reading MS00000927_R1_Cell Membrane.csv...
Reading MS00000929_R0_Cell Membrane.csv...
Reading MS00000929_R1_Cell Membrane.csv...


/tmp/ipykernel_3803/3858020634.py:16: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Reading MS00000931_R0_Cells_Cell Membrane.csv...
Reading MS00000931_R1_Cell Membrane.csv...
Reading MS00000933_R0_Cell Membrane.csv...
Reading MS00000933_R1_Cell Membrane.csv...
Reading MS00000935_R0_Cell Membrane.csv...
Reading MS00000935_R1_Cell Membrane.csv...
Reading MS00000937_R0_Cell Membrane.csv...
Reading MS00000937_R1_Cell Membrane.csv...
Reading MS00000939_R0_Cells_Cell Membrane.csv...
Reading MS00000939_R1_Cell Membrane.csv...
Reading MS00000941_R0_Cell Membrane.csv...
Reading MS00000943_R0_Cell Membrane.csv...
Reading MS00000943_R1_Cell Membrane.csv...
Reading MS00000945_R0_Cell Membrane.csv...
Reading MS00000945_R1_Cell Membrane.csv...
Reading MS00000947_R0_Cell Membrane.csv...
Reading MS00000947_R1_Cell Membrane.csv...
Reading MS00000949_R0_Cells_Cell Membrane.csv...
Reading MS00000949_R1_Cells_Cell Membrane.csv...
Reading MS00000951_R0_Cell Membrane.csv...
Reading MS00000951_R1_Cell Membrane.csv...
Reading MS00000953_R0_Cell Membrane.csv...
Reading MS00000953_R1_Cell Mem

In [5]:
adata

AnnData object with n_obs × n_vars = 3205629 × 53
    obs: 'cell_id', 'sample_id', 'Timeframe', 'batch_id', 'x', 'y', 'Outline Area (µm²)', 'Perimeter (µm)', 'Circularity'
    var: 'marker_name'
    layers: 'intensity_mean', 'intensity_median', 'intensity_std'

In [8]:
#COMMENT/EXPLAIN
all_pstat_results = assess_all_pstat_markers(adata, sample_col='marker_name')

Found 3 pSTAT markers:
  - pSTAT3Y705-D3A7AF555
  - pSTAT1Ty70158D6AF647
  - pSTAT1s727-D3B7AF647

############################################################

MARKER COMPLETENESS ASSESSMENT: pSTAT3Y705-D3A7AF555
Layer used: intensity_mean

Overall Statistics:
  Total cells:      3,104,607
  Zero values:              0 (0.00%)
  NaN values:       3,104,607 (100.00%)
  Negative values:          0
  Positive values:          0 (0.00%)

############################################################

MARKER COMPLETENESS ASSESSMENT: pSTAT1Ty70158D6AF647
Layer used: intensity_mean

Overall Statistics:
  Total cells:      3,104,607
  Zero values:            392 (0.01%)
  NaN values:               8 (0.00%)
  Negative values:          0
  Positive values:  3,104,207 (99.99%)

Positive Value Statistics:
  Min:    0.0093
  Max:    65514.1346
  Mean:   315.0810
  Median: 265.0580

############################################################

MARKER COMPLETENESS ASSESSMENT: pSTAT1s727-D3B7AF647
Lay